## <font color='Red'>MID-CAREER SALARY PREDICTION FOR EACH UNIVERSITY</font>

### For further information about the notebook, the details about each step and other used techniques, click at the following link https://github.com/Iron486/Simple_regression_exercise and check the README.md file.

### <font color='green'>IMPORT LIBRARIES</font> 

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np 
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from pathlib import Path
from sklearn.metrics import mean_absolute_error
import joblib
from sklearn.metrics import mean_squared_error
forest_reg=RandomForestRegressor()
from sklearn import svm
import os
import seaborn as sns
from sklearn.linear_model import LinearRegression


### <font color='green'>READ THE FILES</font> 

In [39]:
diversity_school=pd.read_csv(r"../input/college-tuition-diversity-and-pay/diversity_school.csv") #less important
historical_tuition=pd.read_csv("../input/college-tuition-diversity-and-pay/historical_tuition.csv") #less important
salary_potential=pd.read_csv("../input/college-tuition-diversity-and-pay/salary_potential.csv")
tuition_cost=pd.read_csv("../input/college-tuition-diversity-and-pay/tuition_cost.csv")
tuition_income=pd.read_csv("../input/college-tuition-diversity-and-pay/tuition_income.csv") #less important

### <font color='green'>MERGE THE DATABASES</font> 

In [40]:
merged_salary_potential_tuition_cost=pd.merge(salary_potential,tuition_cost,on='name')
merged_salary_potential_tuition_cost

### <font color='green'>FEATURE EXTRACTION</font> 

In [41]:
merged_salary_potential_tuition_cost['degree_length'].value_counts()
merged_salary_potential_tuition_cost[merged_salary_potential_tuition_cost['degree_length']=='2 Year']
corr_matr=merged_salary_potential_tuition_cost.corr()
corr_matr['mid_career_pay'].sort_values(ascending=False)

In [42]:
merged_salary_potential_tuition_cost.drop(['rank','state_code','state','degree_length'],axis=1,inplace=True)
scatter_matrix(merged_salary_potential_tuition_cost[['mid_career_pay','early_career_pay','out_of_state_total','out_of_state_tuition','room_and_board',
                                                    'stem_percent','in_state_total','in_state_tuition','make_world_better_percent']],figsize=(55,40))

In [43]:

numerical_data_and_type=merged_salary_potential_tuition_cost[['mid_career_pay','early_career_pay','out_of_state_total','out_of_state_tuition','room_and_board',
                                                    'stem_percent','in_state_total','in_state_tuition','make_world_better_percent','type']]

sn.pairplot(numerical_data_and_type,y_vars=['mid_career_pay'], x_vars=['early_career_pay','out_of_state_total','out_of_state_tuition','room_and_board',
                                                    'stem_percent','in_state_total','in_state_tuition','make_world_better_percent']
            , hue='type',size=5) #use x_vars, y_vars to plot only one column
#height=10, aspect=0.2
plt.figure(figsize=(40,30))

In [44]:
#separate 2 datasets
cat_features=merged_salary_potential_tuition_cost['type']
cat_features
cat_features_1hot = pd.get_dummies(cat_features, columns=['type'])#, prefix = ['type'])
cat_features

In [45]:
merged_salary_potential_tuition_cost.drop(['name','state_name','type'],axis=1,inplace=True)


### <font color='green'>HANDLING CATEGORICAL FEATURES</font> 

In [46]:
cat_encoder=OneHotEncoder()
cat_features_1hot=cat_encoder.fit_transform(cat_features.values.reshape(-1, 1))#.to_array()

cat_features_1hot

### <font color='green'>HANDLING MISSING VALUES</font> 

In [47]:
merged_salary_potential_tuition_cost

In [48]:

imputer=IterativeImputer(estimator=BayesianRidge())
num_features1=imputer.fit_transform(merged_salary_potential_tuition_cost) #THE OUTPUT IS AN ARRAY!
num_features1=pd.DataFrame(num_features1,columns=merged_salary_potential_tuition_cost.columns,
                         index=merged_salary_potential_tuition_cost.index)

In [49]:
dataset=pd.concat([num_features1,pd.DataFrame(cat_features_1hot)],axis=1)
train_set, test_set=train_test_split(dataset, test_size=0.2, random_state=42)

In [50]:
 #remember axis=1
train_set_target=train_set.loc[:,'mid_career_pay']
train_set=train_set.drop('mid_career_pay', axis=1)
train_set

In [51]:
 #remember axis=1
test_set_target=test_set.loc[:,'mid_career_pay']
test_set=test_set.drop('mid_career_pay', axis=1)
test_set

### <font color='green'>FEATURE SCALING</font> 

In [52]:
scaler=StandardScaler()
train_set_stand=scaler.fit_transform(train_set.iloc[:,:7])
train_set_features=pd.DataFrame(train_set_stand,columns=train_set.iloc[:,:7].columns, index=train_set.iloc[:,:7].index )
train_set=pd.concat([train_set_features,train_set_target],axis=1)

In [53]:
test_set_stand=scaler.fit_transform(test_set.iloc[:,:7])
test_set_features=pd.DataFrame(test_set_stand,columns=test_set.iloc[:,:7].columns, index=test_set.iloc[:,:7].index )
test_set=pd.concat([test_set_features,test_set_target],axis=1)

In [54]:
train_set_features

### <font color='green'>TRAINING AND PREDICTION</font> 

In [55]:

#train_set_features,train_set_target,test_set_features,test_set_target: train and test sets, target and features sets
#model: model used (for instance, forest_reg, lin_reg, clf etc...)
#file name= name of the file if path=False (or directory with name if path ='directory_name/')
#second name = if there are other particular parameters that have been changed or other characteristics
#that distinguish the model from the similar ones
#path: name of the path. If you want to save the file in the same fold of the notebook, put False 
def func_machine_learning_model(train_set_features,train_set_target,test_set_features,
                                test_set_target,model,file_name,second_name,path):   
    model.fit(train_set_features,train_set_target)    
    train_predictions=model.predict(train_set_features) #predictions on train set
    #rmse train
    mse_train=mean_squared_error(train_set_target,train_predictions)
    rmse_train=np.sqrt(mse_train)
    #mape train
    mape_train_list=[]
    for y_true, y_pred in zip(train_set_target,train_predictions):
        mape_train=np.abs((y_true - y_pred) / y_true)
        mape_train_list.append(mape_train)
    mape_train=np.mean(mape_train_list)*100
    test_predictions=model.predict(test_set_features) #predictions on test set  
    #mae train
    mae_train=mean_absolute_error(train_set_target,train_predictions)
    #-----------------------------------------------------------------------
    #rmse test
    mse_test=mean_squared_error(test_set_target,test_predictions)
    rmse_test=np.sqrt(mse_test)   
    #mape test
    mape_test_list=[]
    for y_true, y_pred in zip(test_set_target,test_predictions):
        mape_test=np.abs((y_true - y_pred) / y_true)
        mape_test_list.append(mape_test)
    mape_test=np.mean(mape_test_list)*100
    list_model= str(model).split('(')
    #mae test
    mae_test=mean_absolute_error(test_set_target,test_predictions)
    #----------------------------------------------------------------------
    
    if path ==False:      
        #save the results on a .txt file
        file1 = open(f"results_{list_model[0]}_{file_name}_{second_name}.txt","w+")  
        file1.write(f'rmse train:{rmse_train}, rmse test:{rmse_test} \n mae train:{mae_train}, mae test:{mae_test} \n mape train:{mape_train}, mape test:{mape_test}')
        file1.close() 

        # save the model on a disk
        filename = f'{list_model[0]}_{file_name}_{second_name}.sav'
        joblib.dump(model, filename)
    else:
        path_new = os.path.join(path, 'results/')
        os.mkdir(path_new)
        file1 = open(f"{path_new}{list_model[0]}_{file_name}_{second_name}.txt","w+")  
        file1.write(f'rmse train:{rmse_train}, rmse test:{rmse_test} \n mae train:{mae_train}, mae test:{mae_test} \n mape train:{mape_train}, mape test:{mape_test}')
        file1.close() 

        # save the model on a disk
        filename = f'{path}{list_model[0]}_{file_name}_{second_name}.sav'
        joblib.dump(model, filename)
    #print(set(zip(test_predictions,test_set_target))) activate if you want to see all the predicted and actual values
    print(f'rmse train:{rmse_train}, rmse test:{rmse_test} \n')
    print(f'mae train:{mae_train}, mae test:{mae_test}')
    print(f'mape train:{mape_train}, mape test:{mape_test}')
    
    
    #########PLOTS
    
    fig, ax = plt.subplots(6,1,figsize=(10,25))
    
    
    ax[0].scatter(train_set_target,train_predictions)
    ax[0].plot([60000,160000],[60000,160000],color='red')
    #ax[0].fill_between(train_set_target,train_predictions-np.percentile(train_predictions, 0.5),train_predictions+np.percentile(train_predictions, 0.5), color=grays, alpha = alpha, label=f"{quantiles}")
    ax[0].set_xlabel('Actual Value')
    ax[0].set_ylabel('Predicted Value')
    ax[0].set_title('Train Dataset:Actual vs Predicted')
    
    ax[1].scatter(test_set_target,test_predictions)
    ax[1].plot([60000,160000],[60000,160000],color='red')
    #ax[0].fill_between(test_set_target,test_predictions-np.percentile(test_predictions, 0.5),test_predictions+np.percentile(test_predictions, 0.5), color=grays, alpha = alpha, label=f"{quantiles}")
    test_predictions
    ax[1].set_title('Test Dataset: Actual vs Predicted')
    ax[1].set_xlabel('Actual Value')
    ax[1].set_ylabel('Predicted Value')
    
    
    ax[2].scatter(train_set_target,train_set_target-train_predictions,color='rebeccapurple')  
    ax[2].plot([60000,160000],[0,0],color='red')
    ax[2].set_title('Residual Train Dataset Plot')
    ax[2].set_ylim(-65000,65000)   
    ax[2].set_ylabel('Residuals')
    ax[2].set_xlabel('Actual Values')
    
    ax[3].scatter(test_set_target,test_set_target-test_predictions,color='rebeccapurple')
    ax[3].plot([60000,160000],[0,0],color='red')
    ax[3].set_title('Residual Test Dataset Plot')
    ax[3].set_ylim(-65000,65000)    
    ax[3].set_ylabel('Residuals')
    ax[3].set_xlabel('Actual Values')
    
    
    ax[4].hist(train_set_target-train_predictions,bins=28,color='orange')
    ax[4].set_title('Residual Train Dataset Histogram') 
    ax[4].plot([0,0],[0,80],color='red')
    ax[4].set_xlim(-65000,65000)
    ax[4].set_xlabel('Residuals')
    ax[4].set_ylabel('Frequency')
    
    ax[5].hist(test_set_target-test_predictions,bins=28,color='orange')
    ax[5].set_title('Residual Test Dataset Histogram')  
    ax[5].plot([0,0],[0,17],color='red')
    ax[5].set_xlim(-65000,65000)
    ax[5].set_xlabel('Residuals')
    ax[5].set_ylabel('Frequency')
    
    
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
    
    
    return rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test

In [56]:
func_machine_learning_model(train_set_features,train_set_target,test_set_features,test_set_target,forest_reg,'second_model','without_make_world_better',path=False)

### <font color='green'>FINAL MODEL</font> 

In [57]:
forest_reg.get_params()